### Feature Engineering

En este notebook se utilizan distintas técnicas para aumentar/reducir la dimensionalidad del dataset de entrenamiento

Técnicas:
    
    - PCA: principal component analysis
    - GridSearch y ML de ensemble

### Load libraries

#### Base Imports

In [2]:
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

#### ML imports

In [3]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, ExtraTreesRegressor

from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.linear_model import Ridge, RidgeCV, BayesianRidge
from sklearn.linear_model import HuberRegressor, TheilSenRegressor, RANSACRegressor

from sklearn.neural_network import MLPRegressor

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler

from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.metrics import r2_score, explained_variance_score

#### ----------------------------> AUSOL ----------------------------------------------------------------------------------------------------------------------------------------------
#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Estudio 0 - Importancia de features sobre un modelo ML optimizado de tipo ensemble

In [ ]:
### --> Base program parameters 

ml_model_folder = "C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\models\\01_horus_traffic_prediction\\NORTE_LITORAL\\machine_learning"
regr_name = "extra_trees_2a-6m4c4v4g"


cluster_features = ['ID_SEGMENT']#, 'COD_LABORALIDAD', 'MES']
excluded_features = ['TIME','FECHA']
forecast_horizon = [15, 60, 120] 


In [ ]:
# Build the training set dictionary

excluded_features.extend(cluster_features)

import itertools
import copy 
clusters_codes = [element for element in itertools.product(*list(map(lambda x: train_data[x].unique().tolist(), cluster_features)))]


if len(cluster_features)==1:
    clusters_codes = [code[0] for code in clusters_codes ]
    
#print(clusters_codes) 

training_sets = dict.fromkeys(clusters_codes) # key: expert_code | value: train_set

# fill the training sets Python dict
for expert_code in training_sets:
    df_train = train_data  # restore pandas dataframe

    if len(cluster_features)==1:
        df_train = df_train.loc[df_train[str(cluster_features[0])]==expert_code]
    else:
        for key, value in zip(cluster_features, expert_code):
            df_train = copy.deepcopy(df_train.loc[df_train[key]==value])
    
    training_sets[expert_code] = copy.deepcopy(df_train)

In [ ]:
# Model restoring
import os
import pickle
from sklearn.externals import joblib


mlmodels_dict = { 'extra_trees_2a-6m4c4v4g':{'is_active': True, 'train_model': True, 'method': ExtraTreesRegressor(min_samples_leaf=10) }, 
}

for regr_name, regr_properties in mlmodels_dict.items():
    if regr_properties['is_active']==True:
        for cluster_code, trainset in training_sets.items():
            for horizon in forecast_horizon:
                model_file = regr_name + '_' + str(cluster_code) + '_' + str(horizon) + ".sav" 
                model_path = os.path.join(ml_model_folder,regr_name, model_file)
                model_file  = pickle.load( open(model_path, "rb" ) ) 
                
                mlmodels_dict[regr_name][str(cluster_code) + '_' + str(horizon)] = model_file

In [ ]:
# sanity test
mlmodels_dict[regr_name]

In [ ]:
import numpy as np

for regr_name, regr_properties in mlmodels_dict.items():
    if regr_properties['is_active']==True:
        for expert_code, trainset in training_sets.items():
            for horizon in forecast_horizon:
                
                
                fet_ind = np.argsort(loaded_model.feature_importances_)[::-1]
                fet_imp = loaded_model.feature_importances_[fet_ind]

#### Estudio 1 - Importancia de features temporales de segmentos contiguos respecto al segmento de estudio